In [ ]:
# Imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import einops
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import export_text
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
import sklearn
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from typing import Callable

import circuits.utils as utils
import circuits.othello_utils as othello_utils
from circuits.eval_sae_as_classifier import construct_othello_dataset

# Setup
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
torch.set_grad_enabled(False)

# Model and data loading
def load_model_and_data(model_name: str, dataset_size: int, custom_functions: list[Callable]):
    model = utils.get_model(model_name, device)
    data = construct_othello_dataset(
        custom_functions=custom_functions,
        n_inputs=dataset_size,
        split="train",
        device=device,
    )
    return model, data

# Cache Neuron Activations
def cache_neuron_activations(model, data: dict, layers: list, batch_size: int, n_batches: int):
    neuron_acts = defaultdict(list)

    for batch_idx in range(n_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        data_batch = data["encoded_inputs"][batch_start:batch_end]
        data_batch = torch.tensor(data_batch, device=device)

        with torch.no_grad(), model.trace(data_batch, scan=False, validate=False):
            for layer in layers:
                neuron_activations_BLD = model.blocks[layer].mlp.hook_post.output.save()
                neuron_acts[layer].append(neuron_activations_BLD)

    for layer in neuron_acts:
        neuron_acts[layer] = torch.stack(neuron_acts[layer])
        neuron_acts[layer] = einops.rearrange(neuron_acts[layer], "n b l c -> (n b) l c")
    
    return neuron_acts

def calculate_binary_activations(neuron_acts: dict, threshold: float=0.1):
    binary_acts = {}

    for layer in neuron_acts:
        max_activations_D = torch.full((2048,), float("-inf"), device=device)

        neuron_acts_BLD = neuron_acts[layer]
        neuron_acts_BD = einops.rearrange(neuron_acts_BLD, "b l d -> (b l) d")

        max_activations_D = torch.max(max_activations_D, neuron_acts_BD.max(dim=0).values)

        binary_acts[layer] = (neuron_acts[layer] > (threshold * max_activations_D)).int()
    return binary_acts

# Prepare data for modeling
def prepare_data(games_BLC: torch.Tensor, mlp_acts_BLD: torch.Tensor):
    X = einops.rearrange(games_BLC, "b l c -> (b l) c").cpu().numpy()
    y = einops.rearrange(mlp_acts_BLD, "b l d -> (b l) d").cpu().numpy()
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate models
def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return model, mse, r2

def calculate_neuron_metrics(model, X, y):
    y_pred = model.predict(X)
    
    # Calculate MSE for all neurons at once
    mse_list = np.mean((y - y_pred)**2, axis=0)
    
    # Calculate R2 for all neurons at once
    ss_res = np.sum((y - y_pred)**2, axis=0)
    ss_tot = np.sum((y - np.mean(y, axis=0))**2, axis=0)
    r2_list = 1 - (ss_res / ss_tot)
    
    return mse_list, r2_list

def calculate_binary_metrics(model, X, y):
    y_pred = model.predict(X)
    
    # Compute true positives, false positives, true negatives, false negatives
    tp = np.sum((y_pred == 1) & (y == 1), axis=0)
    fp = np.sum((y_pred == 1) & (y == 0), axis=0)
    tn = np.sum((y_pred == 0) & (y == 0), axis=0)
    fn = np.sum((y_pred == 0) & (y == 1), axis=0)
    
    # Compute metrics
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = np.divide(tp, tp + fp, out=np.zeros_like(tp, dtype=float), where=(tp + fp) != 0)
    recall = np.divide(tp, tp + fn, out=np.zeros_like(tp, dtype=float), where=(tp + fn) != 0)
    
    # Compute F1 score
    f1 = np.divide(2 * precision * recall, precision + recall, 
                   out=np.zeros_like(precision, dtype=float), 
                   where=(precision + recall) != 0)
    
    return accuracy, precision, recall, f1

# Print decision tree rules
def print_decision_tree_rules(model, feature_names, neuron_index, max_depth=None):
    tree = model.estimators_[neuron_index]
    if feature_names is None:
        feature_names = [f"feature_{i}" for i in range(tree.n_features_in_)]
    tree_rules = export_text(tree, feature_names=feature_names, max_depth=max_depth)
    print(f"Decision Tree Rules for Neuron {neuron_index}:")
    print(tree_rules)

def rc_to_square_notation(row, col):
    letters = "ABCDEFGH"
    letter = letters[row]
    number = 8 - col
    # letter = letters[col]
    return f"{letter}{number}"

def idx_to_square_notation(idx):
    row = idx // 8
    col = idx % 8
    square = rc_to_square_notation(row, col)
    return square

In [ ]:


def process_layer(layer: int, games_BLC: torch.Tensor, neuron_acts: dict, binary_acts: dict, linear_reg: bool = False) -> dict:

    print(f"\nLayer {layer}")
    X_train, X_test, y_train, y_test = prepare_data(games_BLC, neuron_acts[layer])
        
    
    # Decision Tree
    dt_model, dt_mse, dt_r2 = train_and_evaluate(
        MultiOutputRegressor(DecisionTreeRegressor(random_state=42, max_depth=5)),
        X_train, X_test, y_train, y_test
    )

    # Binary Decision Tree
    X_binary_train, X_binary_test, y_binary_train, y_binary_test = prepare_data(games_BLC, binary_acts[layer])
    dt_binary_model = MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth=5))
    dt_binary_model.fit(X_binary_train, y_binary_train)
    y_pred = dt_binary_model.predict(X_binary_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_binary_test, y_pred)
    precision = precision_score(y_binary_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_binary_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_binary_test, y_pred, average='weighted', zero_division=0)

    
    
    layer_results = {
        'layer': layer,
        'regular_dt': {'model': dt_model, 'mse': dt_mse, 'r2': dt_r2},
        'binary_dt': {'model': dt_binary_model, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}
    }

    if linear_reg:
        lasso_model, lasso_mse, lasso_r2 = train_and_evaluate(
            Lasso(alpha=0.005), X_train, X_test, y_train, y_test
        )
        layer_results['lasso'] = {'model': lasso_model, 'mse': lasso_mse, 'r2': lasso_r2}

    return layer_results



In [ ]:

model_name = "Baidicoot/Othello-GPT-Transformer-Lens"
batch_size = 10
n_batches = 10
dataset_size = batch_size * n_batches
layers = list(range(8))

custom_functions = [
    othello_utils.games_batch_to_input_tokens_classifier_input_BLC,
    othello_utils.games_batch_to_board_state_and_input_tokens_classifier_input_BLC,
    othello_utils.games_batch_to_board_state_classifier_input_BLC,
]

model, data = load_model_and_data(model_name, dataset_size, custom_functions)
neuron_acts = cache_neuron_activations(model, data, layers, batch_size, n_batches)
binary_acts = calculate_binary_activations(neuron_acts)
# games_BLC = data[othello_utils.games_batch_to_classifier_input_BLC.__name__]


neuron_acts = utils.to_device(neuron_acts, "cpu")
binary_acts = utils.to_device(binary_acts, "cpu")

In [ ]:
# Use all available cores, but max out at 10
num_cores = min(10, multiprocessing.cpu_count())

results = {}

for layer in layers:
    results[layer] = {}

for custom_function in custom_functions:

    print(f"\n{custom_function.__name__}")
    games_BLC = data[custom_function.__name__]
    games_BLC = utils.to_device(games_BLC, "cpu")

    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        future_to_layer = {executor.submit(process_layer, layer, games_BLC, neuron_acts, binary_acts): layer for layer in layers}
        for future in concurrent.futures.as_completed(future_to_layer):
            layer_result = future.result()
            layer = layer_result['layer']
            results[layer][custom_function.__name__] = {
                'decision_tree': layer_result['regular_dt'],
                'binary_decision_tree': layer_result['binary_dt']
            }

# Print or process results
# for layer, layer_results in results.items():
#     print(f"\nLayer {layer} Results:")
#     print(f"Regular Decision Tree - MSE: {layer_results['decision_tree']['mse']:.4f}, R2: {layer_results['decision_tree']['r2']:.4f}")
#     print(f"Binary Decision Tree - Accuracy: {layer_results['binary_decision_tree']['accuracy']:.4f}, F1: {layer_results['binary_decision_tree']['f1']:.4f}")

In [ ]:
# Function to view metrics for a specific layer
def view_layer_metrics(
    layer: int,
    threshold: float,
    data: dict,
    custom_function_name: str,
    neuron_acts: dict,
    results: dict,
    linear_reg: bool = False,
):
    if layer not in results:
        print(f"Layer {layer} not found in results.")
        return
    
    games_BLC = data[custom_function_name]

    print(f"\n\nMetrics for Layer {layer}:")

    if linear_reg:
        print("Linear Regression (Lasso):")
        print(f"  MSE: {results[layer][custom_function_name]['lasso']['mse']}")
        print(f"  R2: {results[layer][custom_function_name]['lasso']['r2']}")

        lasso_mse, lasso_r2 = calculate_neuron_metrics(
            results[layer][custom_function_name]["lasso"]["model"], X_test, y_test
        )

        print(f"Lasso - Mean MSE: {np.mean(lasso_mse)}, Mean R2: {np.mean(lasso_r2)}")

        linear_r2_tensor = torch.tensor(lasso_r2)
        good_lasso_r2 = (linear_r2_tensor > threshold).sum().item()
        print(f"Number of neurons with R2 > {threshold} (Lasso): {good_lasso_r2}")

    print("Decision Tree:")
    # print(f"  MSE: {results[layer][custom_function_name]['decision_tree']['mse']}")
    # print(f"  R2: {results[layer][custom_function_name]['decision_tree']['r2']}")

    X_train, X_test, y_train, y_test = prepare_data(games_BLC, neuron_acts[layer])

    # Calculate and print neuron-specific metrics
    dt_mse, dt_r2 = calculate_neuron_metrics(
        results[layer][custom_function_name]["decision_tree"]["model"], X_test, y_test
    )

    # print("\nNeuron-specific metrics:")
    # print(f"Decision Tree - Mean MSE: {np.mean(dt_mse)}, Mean R2: {np.mean(dt_r2)}")

    dt_r2_tensor = torch.tensor(dt_r2)

    good_dt_r2 = (dt_r2_tensor > threshold).sum().item()
    print(f"Number of neurons with R2 > {threshold} (Decision Tree): {good_dt_r2}")

    X_binary_train, X_binary_test, y_binary_train, y_binary_test = prepare_data(
        games_BLC, binary_acts[layer]
    )
    accuracy, precision, recall, f1 = calculate_binary_metrics(
        results[layer][custom_function_name]["binary_decision_tree"]["model"], X_binary_test, y_binary_test
    )

    print("\nBinary Decision Tree Metrics:")
    # print(f"  Accuracy: {np.mean(accuracy)}")
    # print(f"  Precision: {np.mean(precision)}")
    # print(f"  Recall: {np.mean(recall)}")
    # print(f"  F1: {np.mean(f1)}")

    good_f1 = (torch.tensor(f1) > threshold).sum().item()
    print(f"Number of neurons with F1 > {threshold} {good_f1}")

    return good_f1

accuracy_by_layer = {}
thresholds = [0.7, 0.8, 0.9]

for custom_function in custom_functions:
    print(f"\n\n{custom_function.__name__}\n")
    accuracy_by_layer[custom_function.__name__] = {}
    for threshold in thresholds:
        print(f"\nThreshold: {threshold}")
        accuracy_by_layer[custom_function.__name__][threshold] = []
        for layer in layers:
            f1_count = view_layer_metrics(layer, threshold, data, custom_function.__name__, neuron_acts, results)
            accuracy_by_layer[custom_function.__name__][threshold].append(f1_count)

In [ ]:
import matplotlib.pyplot as plt

# Assuming layers, custom_functions, and accuracy_by_layer are defined

function_labels = {othello_utils.games_batch_to_input_tokens_classifier_input_BLC: "Input Tokens",
                   othello_utils.games_batch_to_board_state_and_input_tokens_classifier_input_BLC: "Board State and Input Tokens",
                   othello_utils.games_batch_to_board_state_classifier_input_BLC: "Board State"}

colors = ['b', 'g', 'r']  # Colors for different thresholds
markers = ['o', 's', '^']  # Markers for different thresholds

for custom_function in custom_functions:
    function_name = custom_function.__name__
    
    plt.figure(figsize=(12, 7))
    
    for i, threshold in enumerate(thresholds):
        f1_counts = accuracy_by_layer[function_name][threshold]
        
        plt.plot(layers, f1_counts, color=colors[i], marker=markers[i], 
                 label=f'Threshold: {threshold}', linewidth=2, markersize=8)
        
        # Optionally, add value labels on each point
        for j, count in enumerate(f1_counts):
            plt.annotate(str(count), (layers[j], count), 
                         textcoords="offset points", xytext=(0,5), ha='center', 
                         fontsize=8, color=colors[i])
    
    plt.title(f'Number of Neurons with F1 > Threshold for\n{function_labels[custom_function]} by Layer', fontsize=14)
    plt.xlabel('Layer Number', fontsize=12)
    plt.ylabel('F1 Count', fontsize=12)
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig(f'{function_name}_f1_count_by_layer_all_thresholds.png', dpi=300, bbox_inches='tight')
    plt.show()

print("All graphs have been created and saved.")

In [ ]:
layer = 1
neuron_idx = 421
custom_function_name = custom_functions[0].__name__
layer_dt = results[layer][custom_function_name]['decision_tree']['model']
layer_dt = results[layer][custom_function_name]['binary_decision_tree']['model']

games_BLC = data[custom_function_name]

feature_names = []

X_binary_train, X_binary_test, y_binary_train, y_binary_test = prepare_data(
    games_BLC, binary_acts[layer]
)
accuracy, precision, recall, f1 = calculate_binary_metrics(
    results[layer][custom_function_name]["binary_decision_tree"]["model"], X_binary_test, y_binary_test
)

print(f"Neuron 421 F1: {f1[neuron_idx]}")

for i in range(games_BLC.shape[2]):
    if i < 64:
        square = idx_to_square_notation(i)
        feature_names.append(f"Input_{square}")
    elif i < 128:
        j = i - 64
        square = idx_to_square_notation(j)
        feature_names.append(f"Occupied_{square}")
    else:
        feature_names.append(f"Output_{i}")

print_decision_tree_rules(layer_dt, feature_names=feature_names, neuron_index=neuron_idx, max_depth=5)

